### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using aql.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Runing aql in iJava
We can run aql commands in a file. No output will be displayed. 
In _trucate_test.aql_, we have a single AQL command: TRUNCATE test



In [2]:
%sh aql -c "truncate test" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>6.1.0</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [4]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

import com.aerospike.client.query.IndexType;
import com.aerospike.client.query.IndexCollectionType;
import com.aerospike.client.task.IndexTask;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;
import com.aerospike.client.cdt.CTX;

import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.query.RecordSet;

In [5]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);


In [6]:
//Insert a record using a class based construct
class userProfile {
  public void createProfile (String ns, String setName, String AAID, String v1, String v2, String v3, boolean bCheck) {
    Key user = new Key(ns, setName, AAID);      

    List<Value> values = new ArrayList<Value>();
    values.add(Value.get(v1));
    values.add(Value.get(v2));
    values.add(Value.get(v3));  

    Bin segmentBin = new Bin("segment", Value.get(values));
    WritePolicy wPolicy1 = new WritePolicy();
    client.put(wPolicy1, user, segmentBin);
      
    if(bCheck){
      System.out.println(client.get(null, user));
    }   
  }
}

In [7]:
// convenience function to create an index - essentially a pass-through to the client API
void createIndex(String idxName, String binName, IndexType idxType, IndexCollectionType colType, CTX... ctx) {
    try {
            IndexTask task = client.createIndex(null,
                                       "test",
                                       "testset",
                                       idxName,
                                       binName,
                                       idxType,
                                       colType,
                                       ctx);
            task.waitTillComplete(1000, 0);
        }
        catch (AerospikeException ae) {
            if (ae.getResultCode() != ResultCode.INDEX_ALREADY_EXISTS) {
                throw ae;
            }
        } 
        System.out.format("Created index %s on ns=%s set=%s bin=%s.\n", 
                                    idxName, "test", "testset", binName);
}
createIndex("idx_segment", "segment", IndexType.STRING, IndexCollectionType.LIST, null);

Created index idx_segment on ns=test set=testset bin=segment.


In [8]:
userProfile pf = new userProfile();

In [9]:
//userProfile pf = new userProfile();
pf.createProfile("test", "testset", "6f21", "12", "23","34", true);
pf.createProfile("test", "testset", "6f22", "22", "23","34", true);
pf.createProfile("test", "testset", "6f33", "12", "33","34", true);
pf.createProfile("test", "testset", "6f44", "12", "23","44", true);
pf.createProfile("test", "testset", "6f55", "11", "23","34", true);
pf.createProfile("test", "testset", "6f66", "12", "22","34", true);
pf.createProfile("test", "testset", "6f77", "12", "23","33", true);
pf.createProfile("test", "testset", "6f88", "22", "24","44", true);
pf.createProfile("test", "testset", "6f99", "24", "25","54", true);
pf.createProfile("test", "testset", "6f00", "10", "11","34", true);


(gen:1),(exp:448520722),(bins:(segment:[12, 23, 34]))
(gen:1),(exp:448520722),(bins:(segment:[22, 23, 34]))
(gen:1),(exp:448520722),(bins:(segment:[12, 33, 34]))
(gen:1),(exp:448520722),(bins:(segment:[12, 23, 44]))
(gen:1),(exp:448520722),(bins:(segment:[11, 23, 34]))
(gen:1),(exp:448520722),(bins:(segment:[12, 22, 34]))
(gen:1),(exp:448520722),(bins:(segment:[12, 23, 33]))
(gen:1),(exp:448520722),(bins:(segment:[22, 24, 44]))
(gen:1),(exp:448520722),(bins:(segment:[24, 25, 54]))
(gen:1),(exp:448520723),(bins:(segment:[10, 11, 34]))


In [11]:
WritePolicy wPolicy = new WritePolicy();
QueryPolicy qPolicy = new QueryPolicy();

Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
Filter filter = Filter.contains("segment", IndexCollectionType.LIST,"34");
stmt.setFilter(filter);
RecordSet rs = client.query(qPolicy, stmt);

while(rs.next()){
    Record r = rs.getRecord();     
    System.out.println(r); 
}

(gen:1),(exp:448520722),(bins:(segment:[12, 23, 34]))
(gen:1),(exp:448520722),(bins:(segment:[12, 22, 34]))
(gen:1),(exp:448520722),(bins:(segment:[11, 23, 34]))
(gen:1),(exp:448520722),(bins:(segment:[12, 33, 34]))
(gen:1),(exp:448520722),(bins:(segment:[22, 23, 34]))
(gen:1),(exp:448520723),(bins:(segment:[10, 11, 34]))
